In [9]:
import helper
import problem_unittests as tests
import tensorflow as tf

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

In [10]:
view_sentence_range = (0,10)
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len(set([word for word
                                        in source_text.split()]))))

sentences = source_text.split('\n') # '\n'是回车
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:
                                        view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

In [11]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
target_int_text

[[217,
  138,
  31,
  115,
  123,
  130,
  161,
  260,
  274,
  300,
  279,
  31,
  57,
  346,
  200,
  234,
  1],
 [309, 213, 31, 210, 23, 346, 28, 274, 300, 279, 231, 306, 346, 109, 234, 1],
 [301, 31, 210, 123, 346, 32, 274, 300, 279, 31, 210, 6, 346, 121, 234, 1],
 [309, 213, 31, 115, 283, 346, 121, 274, 300, 279, 180, 23, 346, 17, 234, 1],
 [122, 331, 5, 223, 31, 153, 298, 274, 277, 34, 331, 5, 31, 352, 324, 234, 1],
 [170, 223, 77, 31, 191, 274, 277, 34, 77, 31, 153, 298, 234, 1],
 [193, 31, 240, 346, 135, 274, 277, 279, 31, 210, 23, 346, 28, 234, 1],
 [217, 138, 31, 230, 294, 299, 274, 300, 279, 31, 162, 325, 346, 32, 234, 1],
 [167, 223, 31, 331, 5, 153, 143, 274, 277, 34, 331, 5, 31, 153, 298, 234, 1],
 [309,
  213,
  31,
  115,
  230,
  346,
  30,
  274,
  300,
  279,
  31,
  115,
  6,
  346,
  109,
  234,
  1],
 [352, 63, 31, 170, 331, 5, 76, 313, 274, 277, 352, 209, 31, 34, 331, 151, 1],
 [279, 265, 267, 111, 318, 159, 60, 234, 1],
 [329, 31, 11, 346, 121, 274, 300, 279, 31

In [12]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='target')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    target_sequence_lengthc = tf.placeholder(tf.int32, (None, ), name='target_sequence_length')
    max_target_sequence_length = tf.reduce_max(target_sequence_lengthc)
    source_sequence_length = tf.placeholder(tf.int32, (None, ), name='source_sequence_length')
    
    return (inputs, targets, learning_rate, keep_prob, target_sequence_lengthc, max_target_sequence_length,
            source_sequence_length)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


In [13]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # to make decoder robust, feed target sequence
    # TODO: Implement Function
    target_without_last = tf.strided_slice(target_data, [0, 0], 
                                          [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']),
                          target_without_last], 1)
    
    return dec_input

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


In [14]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    embedded_inputs = tf.contrib.layers.embed_sequence(rnn_inputs, vocab_size=source_vocab_size,
                                    embed_dim=encoding_embedding_size)
    
    def make_cell(rnn_size):
        enc_LSTMcell = tf.contrib.rnn.LSTMCell(rnn_size, 
                        initializer=tf.random_uniform_initializer(-0.1,0.1, seed=2))
        enc_LSTMcell = tf.contrib.rnn.DropoutWrapper(enc_LSTMcell, output_keep_prob=keep_prob)
        return enc_LSTMcell
    
    enc_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    
    enc_output, enc_state = tf.nn.dynamic_rnn(enc_cell, embedded_inputs, 
                                             sequence_length=source_sequence_length,
                                             dtype=tf.float32)
    return enc_output, enc_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


In [15]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, input_keep_prob=keep_prob)
        
    training_helper = tf.contrib.seq2seq.TrainingHelper(
                                              inputs=dec_embed_input,
                                              sequence_length=target_sequence_length,
                                              time_major=False)
        
    training_decoder = tf.contrib.seq2seq.BasicDecoder(
                                            dec_cell,
                                            helper=training_helper,
                                            initial_state=encoder_state,
                                            output_layer=output_layer)
        
    decoder_train_final_outputs, _ = tf.contrib.seq2seq.dynamic_decode(
                                        decoder=training_decoder,
#                                         output_time_major=False,
                                        impute_finished=False,
                                        maximum_iterations=max_summary_length
                                        )
    
    return decoder_train_final_outputs



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


In [16]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function

    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, input_keep_prob=keep_prob)
    
    start_tokens = tf.tile(tf.constant([start_of_sequence_id], dtype=tf.int32), [batch_size], name='statrt_tokens')
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings,
                                             start_tokens,
                                             end_of_sequence_id
                                             )
    
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                    inference_helper,
                                    encoder_state,
                                    output_layer=output_layer
                                                        )
    
    inference_decoder_outputs, _ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                      impute_finished=True,
                                      maximum_iterations=max_target_sequence_length,
                                     )
       
    return inference_decoder_outputs



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


In [17]:
from tensorflow.python.layers.core import Dense
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    
    # Embed the target sequences
    dec_embedding = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_input = tf.nn.embedding_lookup(dec_embedding, dec_input)
    
    # Construct the decoder LSTM cell (just like you constructed the encoder cell above)
    def make_cell(rnn_size):
        dec_LSTMcell = tf.contrib.rnn.LSTMCell(rnn_size, 
                        initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return dec_LSTMcell
    
    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    
    # Create an output layer to map the outputs of the decoder to the elements of our vocabulary
    output_layer = Dense(target_vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    # Use the your decoding_layer_train function to get the training logits.
    with tf.variable_scope("decode"):
        decoder_train_final_outputs = decoding_layer_train(encoder_state, 
                                                           dec_cell,
                                                           dec_input, 
                                                           target_sequence_length,
                                                           max_target_sequence_length, 
                                                           output_layer, 
                                                           keep_prob)
    
    # Use your decoding_layer_infer function to get the inference logits.
    with tf.variable_scope("decode", reuse=True):
        inference_decoder_outputs = decoding_layer_infer(encoder_state, dec_cell, 
                                                         dec_embedding, 
                                                         target_vocab_to_int['<GO>'],
                                                         target_vocab_to_int['<EOS>'],
                                                         max_target_sequence_length,
                                                         target_vocab_size, 
                                                         output_layer, 
                                                         batch_size, 
                                                         keep_prob)
    
    return (decoder_train_final_outputs, inference_decoder_outputs)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


In [18]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sequence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    enc_output, enc_final_state = encoding_layer(input_data,
                                                 rnn_size, 
                                                 num_layers, 
                                                 keep_prob, 
                                                 source_sequence_length, 
                                                 source_vocab_size, 
                                                 enc_embedding_size)
    
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    decoder_train_final_outputs, inference_decoder_outputs = decoding_layer(dec_input, enc_final_state,
                                                                            target_sequence_length, 
                                                                            max_target_sequence_length,
                                                                            rnn_size,
                                                                            num_layers, 
                                                                            target_vocab_to_int, 
                                                                            target_vocab_size,
                                                                            batch_size, 
                                                                            keep_prob, 
                                                                            dec_embedding_size)
    
    return decoder_train_final_outputs, inference_decoder_outputs


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


# Build the Graph

In [19]:
# Number of Epochs
epochs = 10
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 256
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 256
decoding_embedding_size = 256
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.5
display_step = 30

In [20]:
import helper

In [26]:
save_path = 'checkpoint/dev'
(source_int_text, target_int_text), \
(source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

max_target_sequence_lemngth = max([len(sentence) for sentence in target_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()
    
    input_shape = tf.shape(input_data)
    
    train_logits, inference_logits = seq2seq_model(
                                        tf.reverse(input_data, [-1]),
                                        targets,
                                        keep_prob,
                                        batch_size,
                                        source_sequence_length,
                                        target_sequence_length,
                                        max_target_sequence_length,
                                        len(source_vocab_to_int),
                                        len(target_vocab_to_int),
                                        encoding_embedding_size,
                                        decoding_embedding_size,
                                        rnn_size,
                                        num_layers,
                                        target_vocab_to_int)
    
    training_logits = tf.identity(train_logits.rnn_output, name="logits")
    inference_logits = tf.identity(inference_logits.sample_id, name="predictions")
    
    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name="masks")
    
    with tf.name_scope("optimization"):
        cost = tf.contrib.seq2seq.sequence_loss(
                                                training_logits,
                                                targets,
                                                masks)
        optimizer = tf.train.AdamOptimizer(lr)
        
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)